# treeinterpreter examples

#### prologue

In [1]:
# treeinterpreter works on scikit-learn's RF
from treeinterpreter import treeinterpreter as ti, utils

import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.datasets import load_iris
from sklearn.datasets import load_boston

### Train Model - Decision Tree Classification

In [2]:
iris = load_iris()
X, y = iris.data, iris.target
#split into training and test 
X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.33, random_state=0)

# fit the model on the training set
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

### Examine Contributions

In [3]:
# note this is for a single tree
for test_sample in [0, 10, 20]:
    prediction, bias, contributions = ti.predict(model, X_test[test_sample].reshape(1,4))
    print("Class Prediction", prediction)
    print("Bias (trainset prior)", bias)

    # now extract contributions for each instance
    for c, feature in zip(contributions[0], iris.feature_names):
        print(feature, c)
    print("\n")

Class Prediction [[ 0.  0.  1.]]
Bias (trainset prior) [[ 0.34  0.31  0.35]]
sepal length (cm) [ 0.  0.  0.]
sepal width (cm) [ 0.  0.  0.]
petal length (cm) [ 0.         -0.43939394  0.43939394]
petal width (cm) [-0.34        0.12939394  0.21060606]


Class Prediction [[ 0.  0.  1.]]
Bias (trainset prior) [[ 0.34  0.31  0.35]]
sepal length (cm) [ 0.  0.  0.]
sepal width (cm) [ 0.  0.  0.]
petal length (cm) [ 0.         -0.43939394  0.43939394]
petal width (cm) [-0.34        0.12939394  0.21060606]


Class Prediction [[ 0.  0.  1.]]
Bias (trainset prior) [[ 0.34  0.31  0.35]]
sepal length (cm) [ 0.  0.  0.]
sepal width (cm) [ 0.   -0.25  0.25]
petal length (cm) [ 0.          0.43939394 -0.43939394]
petal width (cm) [-0.34       -0.49939394  0.83939394]




### Extra Trees Classifier (look this up)

In [4]:
model = ExtraTreesClassifier(n_estimators=10000, n_jobs=-1, random_state=0)
model.fit(X,y)
model.feature_importances_

array([ 0.09522051,  0.05770829,  0.40146144,  0.44560976])

### RF Classifier

In [5]:
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)

instance = X_test[0].reshape(1, -1)
print(rf.predict_proba(instance))

prediction, bias, contributions = ti.predict(rf, instance)
print("Prediction", prediction)
print("Bias (trainset prior)", bias)
print("Feature contributions:")
for c, feature in zip(contributions[0], 
                             iris.feature_names):
    print(feature, c)

[[ 0.          0.01666667  0.98333333]]
Prediction [[ 0.          0.01666667  0.98333333]]
Bias (trainset prior) [[ 0.364  0.298  0.338]]
Feature contributions:
sepal length (cm) [ 0.          0.00828632 -0.00828632]
sepal width (cm) [ 0.  0.  0.]
petal length (cm) [-0.141      -0.21536858  0.35636858]
petal width (cm) [-0.223      -0.07425108  0.29725108]


### Train Model - Regression

In [6]:
boston = load_boston()
rf = RandomForestRegressor()
X, y = boston.data, boston.target

X_train, y_train = X[:300], y[:300]
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Predict - Sample Values

In [7]:
instances = X[[300, 309]]
print(rf.predict(instances[[0]]))
print(rf.predict(instances[[1]]))

[ 28.88]
[ 22.19]


### Analyze predictions

In [8]:
prediction, bias, contributions = ti.predict(rf, instances)

print("Expect prediction and summed contributions to be equal")
print(prediction)
print(bias + np.sum(contributions, axis=1))

print("\n")

for i in range(len(instances)):
    print("Instance", i)
    print("Bias (trainset mean)", bias[i])
    print("Feature contributions:")
    for c, feature in sorted(zip(contributions[i], 
                                 boston.feature_names), 
                             key=lambda x: -abs(x[0])):
        print(feature, round(c, 2))
    print("-"*20)

Expect prediction and summed contributions to be equal
[ 28.88  22.19]
[ 28.88  22.19]


Instance 0
Bias (trainset mean) 25.3904333333
Feature contributions:
RM 4.25
NOX -0.69
B -0.57
LSTAT 0.47
INDUS 0.34
TAX -0.24
PTRATIO -0.21
CRIM -0.17
DIS 0.14
AGE 0.12
RAD 0.04
ZN 0.0
CHAS 0.0
--------------------
Instance 1
Bias (trainset mean) 25.3904333333
Feature contributions:
RM -5.81
LSTAT 2.97
CRIM 0.66
B -0.49
AGE -0.45
DIS 0.08
TAX -0.07
RAD -0.04
CHAS -0.02
PTRATIO -0.01
INDUS -0.01
ZN 0.0
NOX 0.0
--------------------


### Predictions - Two Subsets

In [9]:
# take two subsets from the data ...
ds1 = X[300:400]
ds2 = X[400:]

# extract predictions and terms
prediction1, bias1, contributions1 = ti.predict(rf, ds1)
prediction2, bias2, contributions2 = ti.predict(rf, ds2)

# and check what the predicted average price is
print(np.mean(prediction1))
print(np.mean(prediction2))

22.4021
18.6062264151


### Bias and Contributions

In [10]:
# biases should be the same for all. It's a prior class difference, or intercept term
print(set(bias1))
print(set(bias2))

# there's a difference in mean contribs between these two datasets
totalc1 = np.mean(contributions1, axis=0) 
totalc2 = np.mean(contributions2, axis=0)

# feature contribution difference should be same as mean difference
print(np.sum(totalc1 - totalc2))
print(np.mean(prediction1) - np.mean(prediction2))

{25.390433333333341}
{25.390433333333341}
3.79587358491
3.79587358491


### Difference in Contribution Between Two Predicted Sets

In [11]:
fdiffs = pd.DataFrame.from_items([('fnames', boston.feature_names)
                , ('totalc1', totalc1)
                , ('totalc2', totalc2)])

fdiffs['diff'] = fdiffs['totalc1'] - fdiffs['totalc2']
fdiffs['absdiff'] = abs(fdiffs['totalc1'] - fdiffs['totalc2'])

fdiffs = fdiffs.sort_values('absdiff', ascending=False).reset_index()

for i in range(fdiffs.shape[0]):
    print(fdiffs['fnames'][i], round(fdiffs['diff'][i], 4))

LSTAT 2.6629
RM 0.5296
CRIM 0.5059
TAX 0.1101
RAD -0.1026
B 0.0612
AGE -0.0482
PTRATIO 0.0413
NOX -0.0388
INDUS 0.0266
DIS 0.0258
CHAS 0.0148
ZN 0.0071


### Looking at Interactions

In [12]:
prediction1, bias1, contributions1 = ti.predict(rf, ds1, joint_contribution=True)
prediction2, bias2, contributions2 = ti.predict(rf, ds2, joint_contribution=True)


print(contributions1)
# totalc1 = np.mean(contributions1, axis=0) 
# totalc2 = np.mean(contributions2, axis=0)

aggregated_contributions1 = utils.aggregated_contribution(contributions1)
aggregated_contributions2 = utils.aggregated_contribution(contributions2)

#print(totalcalc1)
# print(aggregated_contributions1)

[{(5, 6, 7, 8, 9, 12): 0.025000000000000001, (5, 9): -0.083915978994748036, (5, 6): 0.0055549109640892881, (5, 11, 12): -0.15249999999999986, (0, 2, 5, 6, 9, 10, 11): -0.013333333333333286, (0, 4, 5, 10, 11, 12): -0.58062499999999984, (0, 5, 7, 9, 12): -0.086740740740741631, (5, 7, 12): -0.030592105263157521, (5, 6, 11, 12): -0.010000000000000142, (0, 4, 5, 6, 7): 0.098000000000000045, (0, 5, 7, 9, 10, 12): 0.067904761904761912, (5, 6, 7): -0.38414634146341486, (2, 5, 12): 0.1994444444444447, (2, 5, 6, 12): -0.010000000000000142, (0, 5, 6, 7, 9, 10, 12): 0.021428571428571529, (4, 5, 7): -0.11212121212121211, (5, 10): -0.50833333333333319, (0, 5, 6, 9, 10): -0.04781818181818203, (0, 5, 6, 9, 10, 11): -0.13066666666666649, (5, 7): 0.10312121212121177, (5, 6, 7, 9): 0.043095238095237673, (5,): 4.7154626882829422, (5, 6, 7, 9, 12): 0.091428571428571803, (5, 6, 9): -0.11049155145929319, (5, 10, 12): 0.18684124216382364, (0, 5, 10, 11, 12): -0.32293055555555628, (5, 6, 10): 0.158317307692307

### Show that aggregated contributions and overall predictions are equal

In [13]:
print (np.sum(list(aggregated_contributions1.values())) - 
       np.sum(list(aggregated_contributions2.values())))
print (np.mean(prediction1) - np.mean(prediction2))

3.79587358491
3.79587358491


### Find difference in aggregate contributions and print top 10

In [14]:
res = []
for k in set(aggregated_contributions1.keys()).union(
              set(aggregated_contributions2.keys())):
    res.append(([boston["feature_names"][index] for index in k] , 
               aggregated_contributions1.get(k, 0) - aggregated_contributions2.get(k, 0)))   
         
for lst, v in (sorted(res, key=lambda x:-abs(x[1])))[:10]:
    print (lst, v)    

['RM', 'LSTAT'] 2.36927805198
['RM'] 0.736727814862
['CRIM', 'RM', 'LSTAT'] 0.393708653436
['RM', 'PTRATIO', 'LSTAT'] 0.116411222398
['CRIM', 'RM', 'AGE', 'LSTAT'] -0.0812933027248
['RM', 'AGE', 'RAD', 'LSTAT'] -0.0726857389937
['INDUS', 'RM', 'AGE', 'B', 'LSTAT'] 0.0525622101045
['RM', 'TAX', 'LSTAT'] 0.0490630217612
['INDUS', 'RM', 'LSTAT'] 0.048488408206
['CRIM', 'RM', 'PTRATIO', 'LSTAT'] 0.0477103659785
